### PYTHON ДЛЯ DATA SCIENCE
###### ** КУРСОВАЯ РАБОТА.** АКИМОВ Я.В. **ИИ-181**
---
<br>
<br>
<br>

#### I. РАБОТА С ДАННЫМИ

##### **ЗНАКОМСТВО С ДАТАСЕТОМ, КОРРЕКЦИЯ ВЫБРОСОВ.**
<br>

In [1]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from matplotlib import rcParams
import seaborn as sns

%matplotlib inline

In [2]:
df = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
alldata = pd.concat([df, test], sort = False, ignore_index = True)

In [4]:
df.loc[(df.Square > 18) & (df.Square < 250)].groupby('Rooms')['Square'].agg(['mean', 'std'])

,mean,std
Rooms,,
0.0,92.940802,63.340563
1.0,41.325571,9.524270
2.0,56.519501,10.834056
3.0,77.023390,15.479314
4.0,98.377544,23.819556
5.0,113.613157,40.174086
6.0,59.414334,NaN
10.0,59.964120,1.282897
19.0,42.006046,NaN


In [5]:
alldata.loc[(alldata.Square) > 50 | (alldata.Square < 70)].head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,NaN,0,11,B,184966.930730
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644


In [6]:
alldata.loc[alldata.Id.isin([11602]), ['Square', 'LifeSquare']] = 41,38
alldata.loc[alldata.Id.isin([2307]), ['Square', 'LifeSquare']] = 41, 29
alldata.loc[(alldata.Square < alldata.LifeSquare), ['Square', 'LifeSquare']] = alldata[alldata.Square < alldata.LifeSquare][['LifeSquare', 'Square']].values
alldata.loc[alldata.Id == 16550, ['Square', 'LifeSquare']] = alldata.loc[alldata.Id == 16550, ['LifeSquare', 'Square']].values
alldata.loc[alldata.Id == 11533, ['Square', 'LifeSquare']] = 48, 33
alldata.loc[alldata.Id == 15886, ['Square', 'LifeSquare']] = 78, 61
alldata.loc[alldata.Id == 14990, ['Square', 'LifeSquare']] = 63, 48
alldata.loc[alldata.Id == 28, ['Square', 'LifeSquare']] = 60, 40
alldata.loc[alldata.Id == 7917, ['Rooms']] = 6
alldata.loc[(alldata.Rooms < 1) | (alldata.Rooms > 5), 'Rooms'] = 1, 5, 1, 6, 1, 1, 2, 2, 2, 1, 3, 1, 6, 6, 2, 6, 1

In [7]:
alldata.loc[(alldata.Square < alldata.LifeSquare), ['Square', 'LifeSquare']] = \
alldata[alldata.Square < alldata.LifeSquare][['LifeSquare', 'Square']].values

In [8]:
alldata.loc[alldata.Square < 13, ['Square', 'LifeSquare']] = [46, 20], [46, 25], [55, 51], [37, 30], [56, 43], [45, 36], \
                                                            [26, 19], [52, 29], [46, 45], [37, 35], [43, 26], [56, 50], [51, 38], \
                                                            [23, 18], [49, 29], [29, 19]

In [9]:
alldata.loc[alldata['Square'] < 30, ['Rooms']] = 1

In [10]:
alldata.loc[alldata['LifeSquare'] < 10, ['LifeSquare']] = \
alldata.loc[alldata['LifeSquare'] < 10, ['LifeSquare']].values * 10

alldata.loc[alldata['LifeSquare'] < 10, ['LifeSquare']] = \
alldata.loc[alldata['LifeSquare'] < 10, ['LifeSquare']].values * 10

In [11]:
alldata.loc[(alldata.Square < alldata.LifeSquare), ['Square', 'LifeSquare']] = \
alldata[alldata.Square < alldata.LifeSquare][['LifeSquare', 'Square']].values

In [12]:
alldata.loc[alldata['Id'] == 16550, ['Square', 'LifeSquare']] = 81, 74
alldata.loc[alldata['HouseYear'] == 4968, 'HouseYear'] = 1968
alldata.loc[alldata['HouseYear'] == 20052011, 'HouseYear'] = 2005

##### **СОЗДАНИЕ НОВЫХ ПРИЗНАКОВ.**
<br>

In [13]:
meanyear = alldata.groupby(['HouseYear', 'Rooms'], as_index = False).Square.agg({'MeanYear': 'mean'})
meanroom = alldata.groupby(['Rooms'], as_index = False).Square.agg({'MeanRoom': 'mean'})

alldata = pd.merge(alldata, meanyear, on = ['HouseYear', 'Rooms'], how = 'left')
alldata = pd.merge(alldata, meanroom, on = ['Rooms'], how = 'left')

alldata['LifeSquare'] = alldata['LifeSquare'].fillna(alldata['MeanYear'])

distmeanprice = alldata.groupby(['DistrictId', 'Rooms'], as_index = False).Price.agg({'DistMeanPrice': 'mean'})
meanprice = alldata.groupby(['Rooms'], as_index = False).Price.agg({'MeanPrice': 'mean'})
meanprices = pd.merge(distmeanprice, meanprice, on = ['Rooms'], how = 'left')
meanprices['DistMeanPrice'] = meanprices['DistMeanPrice'].fillna(meanprices['MeanPrice'])

alldata = pd.merge(alldata, meanprices, on = ['DistrictId', 'Rooms'], how = 'left')

##### **ПРЕОБРАЗОВАНИЕ НЕЧИСЛОВЫХ ЗНЧАНИЙ.**
<br>

In [14]:
alldata['Healthcare_1'] = alldata.Healthcare_1.fillna(1500)

alldata['Ecology_2'] = (alldata['Ecology_2'] == 'B').astype(int)
alldata['Ecology_3'] = (alldata['Ecology_3'] == 'B').astype(int)
alldata['Shops_2'] = (alldata['Shops_2'] == 'B').astype(int)

##### **РАЗДЕЛЕНИЕ ЕДИНОГО СКОРРЕКТИРОВАННОГО СЕТА НА ИСХОДНЫЕ.**
###### *(СЛИЯНИЕ ПРОИЗВОДИЛОСЬ ДЛЯ УЛУЧШЕНИЯ ГРУППОВЫХ СТАТИСТИК)*
<br>

In [15]:
new_train = alldata.loc[alldata['Id'].isin(df.Id.values)]
new_test = alldata.loc[alldata['Id'].isin(test.Id.values)].drop(columns = ['Price'])

#### II. MODEL

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression as LR
from sklearn.ensemble import RandomForestRegressor as RF

In [17]:
from sklearn.metrics import r2_score as r2

In [18]:
trn, vld = train_test_split(new_train, test_size = .3, random_state = 40)

In [19]:
trn.shape, vld.shape

((7000, 24), (3000, 24))

In [20]:
features = ['KitchenSquare', 'HouseYear', 'Square',
            'Floor', 'HouseFloor', 'LifeSquare',
            'Rooms', 'MeanRoom', 'MeanYear',
            'DistMeanPrice', 'DistrictId', 'Ecology_1',
            'Ecology_2', 'Ecology_3', 'Social_1' ,'Social_2',
            'Social_3', 'Shops_1', 'Shops_2']

In [21]:
lr = LR()
rf = RF(n_estimators = 300, max_depth = 14, min_samples_leaf = 3, min_samples_split=3)

##### **ОТБОР НАИБОЛЕЕ ЗНАЧИМЫХ ФИЧ ЖАДНЫМ АЛГОРИТМОМ.**
<br>

In [22]:
def r2_test(feats):
    lr.fit(trn.loc[:, feats], trn['Price'])
    rf.fit(trn.loc[:, feats], trn['Price'])
    
    pred_trn_lr = lr.predict(trn.loc[:, feats])
    pred_vld_lr = lr.predict(vld.loc[:, feats])
    pred_trn_rf = rf.predict(trn.loc[:, feats])
    pred_vld_rf = rf.predict(vld.loc[:, feats])
    
    
    return {'ptlr': r2(trn['Price'], pred_trn_lr),
            'pvlr': r2(vld['Price'], pred_vld_lr),
            'ptrf': r2(trn['Price'], pred_trn_rf),
            'pvrf': r2(vld['Price'], pred_vld_rf)}

In [23]:
feats = []
f = 0
result = 0
while len(features) > 1:
    flag = False
    for i, feature in enumerate(features):
        feats.append(features[i])
        check_arr_r2 = r2_test(feats)
        if check_arr_r2['pvrf'] > result:
            result = check_arr_r2['pvrf']
            max_index = i
            flag = True
        del feats[-1]
    
    print(f'{feats}')
    print(f"R2 train LR: {check_arr_r2['ptlr']:10.5f}, R2 valid LR: {check_arr_r2['pvlr']:10.5f}, R2 train RF: {check_arr_r2['ptrf']:10.5f}, R2 valid RF: {check_arr_r2['pvrf']:10.5f}")
   
    if flag == False:
        break
    feats.append(features.pop(max_index))
print(feats)

[]
R2 train LR:    0.00447, R2 valid LR:    0.00545, R2 train RF:    0.00447, R2 valid RF:    0.00548
['DistMeanPrice']
R2 train LR:    0.65139, R2 valid LR:    0.63264, R2 train RF:    0.66247, R2 valid RF:    0.57713
['DistMeanPrice', 'Square']
R2 train LR:    0.68394, R2 valid LR:    0.66380, R2 train RF:    0.84608, R2 valid RF:    0.70307
['DistMeanPrice', 'Square', 'MeanYear']
R2 train LR:    0.69544, R2 valid LR:    0.67652, R2 train RF:    0.87351, R2 valid RF:    0.74305
['DistMeanPrice', 'Square', 'MeanYear', 'HouseFloor']
R2 train LR:    0.70351, R2 valid LR:    0.68895, R2 train RF:    0.88342, R2 valid RF:    0.75639
['DistMeanPrice', 'Square', 'MeanYear', 'HouseFloor', 'Social_3']
R2 train LR:    0.70351, R2 valid LR:    0.68885, R2 train RF:    0.89423, R2 valid RF:    0.76541
['DistMeanPrice', 'Square', 'MeanYear', 'HouseFloor', 'Social_3', 'Rooms']
R2 train LR:    0.71045, R2 valid LR:    0.69481, R2 train RF:    0.89546, R2 valid RF:    0.76649
['DistMeanPrice', 'Squa

In [32]:
lr.fit(trn.loc[:, feats], trn['Price'])
rf.fit(trn.loc[:, feats], trn['Price'])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=14,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=3, min_samples_split=3,
           min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [25]:
# feats7704 = feats

# feats7722 = feats

# feats

In [33]:
pred_trn_lr = lr.predict(trn.loc[:, feats])
pred_vld_lr = lr.predict(vld.loc[:, feats])

pred_trn_rf = rf.predict(trn.loc[:, feats])
pred_vld_rf = rf.predict(vld.loc[:, feats])

In [34]:
r2(trn['Price'], pred_trn_lr)

0.7194142032639581

In [35]:
r2(vld['Price'], pred_vld_lr)

0.7032859037692518

In [36]:
r2(trn['Price'], pred_trn_rf)

0.9095950906672069

In [37]:
r2(vld['Price'], pred_vld_rf)

0.7720438611328462

In [31]:
new_test['Price'] = rf.predict(new_test.loc[:, feats])
new_test[['Id','Price']].to_csv('YAkimov_prediction.csv', index = None)